In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/sedroc/Bachelor/FEPCEU')

In [3]:
from tqdm import tqdm

In [4]:
from enhanced_experiment import EnhancedTimeSeriesExperiment
from models.arima import Arima
from models.baseline import Baseline
from models.lgbm import LGBM
from models.neuralproph import Neuralprophet
from models.timegpt import TimeGPT

from data.data_retriever import Dataretreiver

/home/sedroc/Bachelor/FEPCEU/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [5]:
# Define models
models = {
    'BASELINE': Baseline(),
    #'LGBM': LGBM(),
    #'ARIMA': Arima(order=[1,0,1], seasonal_order=[1,1,1,24]),
    #'NEURALPROPHET': Neuralprophet()
    #'TimeGPT': TimeGPT()

}

In [9]:
for i in tqdm(range(1, 10)):
    data = Dataretreiver(reduce='pca_kmeans', debug=False, reduction_num_cols=i)
    df = data.combined

    # Initialize the experiment
    experiment = EnhancedTimeSeriesExperiment(
        models=models,
        target_column='price',
        forecast_horizon=192,
        n_splits=1,
        step_size=192 # 8 days are skipped between each forecast.
    )

    # Run the experiments with these custom combinations
    experiment.run_feature_group_experiments(
        df=df,
        add_all_columns=True,
        add_base_columns=False,
    )

    results_df_MAE = experiment.summarize_feature_group_results(metric='MAE')
    results_df_MSE = experiment.summarize_feature_group_results(metric='RMSE')
    results_df_time = experiment.summarize_feature_group_results(metric='elapsed_time')

    print('='*50)
    print(f"Iterataion {i} results")
    print("MAE:")
    print(results_df_MAE)
    print("RMSE")
    print(results_df_MSE)
    print("Elapsed time")
    print(results_df_time)
    print('='*50)
    
    #experiment.save_feature_group_results_to_csv(f"../results/opt_num_areas/pca_kmeans_{i}.csv")

 11%|█         | 1/9 [00:01<00:10,  1.35s/it]

Identified area columns: {'sun': {21: 'sun_area_21'}, 'wind': {37: 'wind_area_37'}}
Cross-validation splits with sliding window:
Split 1: Train start = 2024-01-01 00:00:00 (index 0), Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531), Training size = 4340

Running experiment for feature group: all_areas
Using columns: ['sun_area_21', 'wind_area_37']
Running CV split 1/1 for feature group all_areas
Iterataion 1 results
MAE:
                    BASELINE
all_areas  260.1990 ± 0.0000
RMSE
                    BASELINE
all_areas  357.9761 ± 0.0000
Elapsed time
                  BASELINE
all_areas  0.0237 ± 0.0000


 22%|██▏       | 2/9 [00:02<00:09,  1.30s/it]

Identified area columns: {'sun': {28: 'sun_area_28', 13: 'sun_area_13'}, 'wind': {17: 'wind_area_17', 24: 'wind_area_24'}}
Cross-validation splits with sliding window:
Split 1: Train start = 2024-01-01 00:00:00 (index 0), Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531), Training size = 4340

Running experiment for feature group: all_areas
Using columns: ['sun_area_28', 'sun_area_13', 'wind_area_17', 'wind_area_24']
Running CV split 1/1 for feature group all_areas
Iterataion 2 results
MAE:
                    BASELINE
all_areas  260.1990 ± 0.0000
RMSE
                    BASELINE
all_areas  357.9761 ± 0.0000
Elapsed time
                  BASELINE
all_areas  0.0217 ± 0.0000


 33%|███▎      | 3/9 [00:03<00:07,  1.30s/it]

...

Identified area columns: {'sun': {22: 'sun_area_22', 7: 'sun_area_7', 9: 'sun_area_9', 39: 'sun_area_39', 11: 'sun_area_11', 25: 'sun_area_25', 32: 'sun_area_32'}, 'wind': {18: 'wind_area_18', 34: 'wind_area_34', 2: 'wind_area_2', 4: 'wind_area_4', 32: 'wind_area_32', 19: 'wind_area_19', 26: 'wind_area_26'}}
Cross-validation splits with sliding window:
Split 1: Train start = 2024-01-01 00:00:00 (index 0), Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531), Training size = 4340

Running experiment for feature group: all_areas
Using columns: ['sun_area_22', 'sun_area_7', 'sun_area_9', 'sun_area_39', 'sun_area_11', 'sun_area_25', 'sun_area_32', 'wind_area_18', 'wind_area_34', 'wind_area_2', 'wind_area_4', 'wind_area_32', 'wind_area_19', 'wind_area_26']
Running CV split 1/1 for feature group all_areas
Iterataion 7 results
MAE:
                    BASELINE
all_areas  260.1990 ± 0.0000
RMSE
                    BASELINE
all_areas  357.9761 ± 0.0000
Elaps

 89%|████████▉ | 8/9 [00:10<00:01,  1.40s/it]

Identified area columns: {'sun': {22: 'sun_area_22', 7: 'sun_area_7', 39: 'sun_area_39', 9: 'sun_area_9', 19: 'sun_area_19', 32: 'sun_area_32', 11: 'sun_area_11', 25: 'sun_area_25'}, 'wind': {22: 'wind_area_22', 35: 'wind_area_35', 7: 'wind_area_7', 9: 'wind_area_9', 11: 'wind_area_11', 32: 'wind_area_32', 25: 'wind_area_25', 31: 'wind_area_31'}}
Cross-validation splits with sliding window:
Split 1: Train start = 2024-01-01 00:00:00 (index 0), Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531), Training size = 4340

Running experiment for feature group: all_areas
Using columns: ['sun_area_22', 'sun_area_7', 'sun_area_39', 'sun_area_9', 'sun_area_19', 'sun_area_32', 'sun_area_11', 'sun_area_25', 'wind_area_22', 'wind_area_35', 'wind_area_7', 'wind_area_9', 'wind_area_11', 'wind_area_32', 'wind_area_25', 'wind_area_31']
Running CV split 1/1 for feature group all_areas
Iterataion 8 results
MAE:
                    BASELINE
all_areas  260.1990 ± 0.000

100%|██████████| 9/9 [00:12<00:00,  1.39s/it]

Identified area columns: {'sun': {23: 'sun_area_23', 7: 'sun_area_7', 39: 'sun_area_39', 9: 'sun_area_9', 16: 'sun_area_16', 31: 'sun_area_31', 25: 'sun_area_25', 32: 'sun_area_32', 6: 'sun_area_6'}, 'wind': {22: 'wind_area_22', 34: 'wind_area_34', 7: 'wind_area_7', 9: 'wind_area_9', 16: 'wind_area_16', 32: 'wind_area_32', 20: 'wind_area_20', 31: 'wind_area_31', 6: 'wind_area_6'}}
Cross-validation splits with sliding window:
Split 1: Train start = 2024-01-01 00:00:00 (index 0), Train end = 2024-06-29 20:00:00 (index 4340), Test end = 2024-07-07 19:00:00 (index 4531), Training size = 4340

Running experiment for feature group: all_areas
Using columns: ['sun_area_23', 'sun_area_7', 'sun_area_39', 'sun_area_9', 'sun_area_16', 'sun_area_31', 'sun_area_25', 'sun_area_32', 'sun_area_6', 'wind_area_22', 'wind_area_34', 'wind_area_7', 'wind_area_9', 'wind_area_16', 'wind_area_32', 'wind_area_20', 'wind_area_31', 'wind_area_6']
Running CV split 1/1 for feature group all_areas
Iterataion 9 resul

In [ ]:
for i in tqdm(range(1, 2)):
    data = Dataretreiver(reduce='pca_pure', debug=True, reduction_num_cols=i)
    df = data.combined

    # Run the experiments with these custom combinations
    experiment.run_feature_group_experiments(
        df=df,
        add_all_columns=True,
        add_base_columns=False,
    )

    results_df_MAE = experiment.summarize_feature_group_results(metric='MAE')
    results_df_MSE = experiment.summarize_feature_group_results(metric='RMSE')
    results_df_time = experiment.summarize_feature_group_results(metric='elapsed_time')

    print('='*50)
    print(f"Iterataion {i} results")
    print(results_df_MAE)
    print(results_df_MSE)
    print(results_df_time)
    print('='*50)
    
    #experiment.save_feature_group_results_to_csv(f"../results/opt_num_areas/pca_pure_{i}.csv")

In [ ]:
for i in tqdm(range(1, 2)):
    data = Dataretreiver(reduce='pearson', debug=True, reduction_num_cols=i)
    df = data.combined

    # Run the experiments with these custom combinations
    experiment.run_feature_group_experiments(
        df=df,
        add_all_columns=True,
        add_base_columns=False,
    )

    results_df_MAE = experiment.summarize_feature_group_results(metric='MAE')
    results_df_MSE = experiment.summarize_feature_group_results(metric='RMSE')
    results_df_time = experiment.summarize_feature_group_results(metric='elapsed_time')

    print('='*50)
    print(f"Iterataion {i} results")
    print(results_df_MAE)
    print(results_df_MSE)
    print(results_df_time)
    print('='*50)
    
    #experiment.save_feature_group_results_to_csv(f"../results/opt_num_areas/pearson_{i}.csv")